Колонки - 'PassengerId', 'HomePlanet', 'Destination', 'Name' были удалены так как не влияют на состояние пассажиров в момент аномалии
Колонки - 'CryoSleep', 'VIP', 'Transported' Подверглись label_encoding'y
Колонка Cabin была разделена на новые колонки, которые были закодированы с помощью one_hot encoding в следствии того, что посадочное место вероятно коррелирует с возможностью повлиять на воздействие эффектов аномалии
Колонки - 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck' являются колонками трат, nan'ы были заполнены с помощью линейной регрессии на основании друг друга
Колонка Age была заполнена средним значение 
Колонки - 'CryoSleep', 'VIP' были заполнены линейной регрессией на основе всех остальных колонок кроме Transported

In [224]:
import pandas as pd
import numpy as np
import scipy as sp

In [225]:
df = pd.read_csv('/home/uncookie/Documents/datasets/spaceship-titanic/train.csv')
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [226]:
del_cols = ['PassengerId', 'HomePlanet', 'Destination', 'Name']

for col in del_cols:
    df = df.loc[:, df.columns != col]

df

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,False,B/0/P,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,False,F/0/S,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,False,A/0/S,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,False,A/0/S,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,False,F/1/S,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...
8688,False,A/98/P,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,True,G/1499/S,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,False,G/1500/S,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,False,E/608/S,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [227]:
df.columns[df.isna().any()].tolist()

['CryoSleep',
 'Cabin',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck']

In [228]:
df.isna().sum()

CryoSleep       217
Cabin           199
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

In [229]:
label_cols = ['CryoSleep', 'VIP', 'Transported']

for col in label_cols:
    df[col] = df[col].replace({True: 1, False: 0})
df

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0.0,B/0/P,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,F/0/S,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1
2,0.0,A/0/S,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0
3,0.0,A/0/S,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0
4,0.0,F/1/S,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...
8688,0.0,A/98/P,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0
8689,1.0,G/1499/S,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8690,0.0,G/1500/S,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1
8691,0.0,E/608/S,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0


In [230]:
df['Cabin'].fillna('123', inplace=True)

df['first_letter'] = df['Cabin'].str[0]
df['second_letter'] = df['Cabin'].str[-1]

df.replace('NA', np.nan, inplace=True)
df['first_letter'].replace('1', np.nan, inplace=True)
df['second_letter'].replace('3', np.nan, inplace=True)
df.drop('Cabin', axis=1, inplace=True)

df

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,first_letter,second_letter
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0,B,P
1,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1,F,S
2,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0,A,S
3,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0,A,S
4,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1,F,S
...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0,A,P
8689,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0,G,S
8690,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1,G,S
8691,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0,E,S


In [231]:
df.isna().sum()

CryoSleep        217
Age              179
VIP              203
RoomService      181
FoodCourt        183
ShoppingMall     208
Spa              183
VRDeck           188
Transported        0
first_letter     199
second_letter    199
dtype: int64

In [232]:
cols_to_label = ['first_letter', 'second_letter']

def one_code(df, col):
    one_hot = pd.get_dummies(df[col])
    df = df.drop(col,axis = 1)
    df = df.join(one_hot)
    return df

for col in cols_to_label:
    df = one_code(df.copy(), col)

df = df.replace({True: 1, False: 0})

df


,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,A,B,C,D,E,F,G,T,P,S
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,1,0
1,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1,0,0,0,0,0,1,0,0,0,1
2,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0,0,0,0,0,0,0,0,1
3,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0,1,0,0,0,0,0,0,0,0,1
4,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0,1,0,0,0,0,0,0,0,1,0
8689,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,1
8690,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1,0,0,0,0,0,0,1,0,0,1
8691,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0,0,0,0,0,1,0,0,0,0,1


In [233]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,A,B,C,D,E,F,G,T,P,S
CryoSleep,1.000000,-0.074273,-0.081402,-0.252396,-0.211510,-0.212514,-0.203991,-0.198857,0.468645,-0.031378,0.126328,0.030447,-0.068608,-0.113267,-0.224790,0.245972,-0.016237,-0.022918,0.015423
Age,-0.074273,1.000000,0.092819,0.068723,0.130421,0.033133,0.123970,0.101007,-0.075026,0.075568,0.104058,0.127957,0.077988,0.028638,-0.018938,-0.212117,0.013673,-0.011880,0.010862
VIP,-0.081402,0.092819,1.000000,0.058785,0.129799,0.018295,0.061059,0.125974,-0.037650,0.134195,0.065237,0.066555,0.068180,-0.013362,-0.058120,-0.099889,-0.003761,0.008714,-0.011010
RoomService,-0.252396,0.068723,0.058785,1.000000,-0.015889,0.054480,0.010080,-0.019581,-0.244611,-0.023269,-0.065617,-0.020310,0.162944,0.044106,0.086898,-0.146530,0.007364,0.007583,-0.015591
FoodCourt,-0.211510,0.130421,0.129799,-0.015889,1.000000,-0.014228,0.221891,0.227995,0.046566,0.119257,0.156435,0.262541,0.020183,-0.039740,-0.133150,-0.151747,0.014134,-0.019812,0.019700
ShoppingMall,-0.212514,0.033133,0.018295,0.054480,-0.014228,1.000000,0.013879,-0.007322,0.010141,-0.017418,-0.010739,0.000934,0.055252,0.031655,0.074909,-0.109245,-0.006222,0.022192,-0.019041
Spa,-0.203991,0.123970,0.061059,0.010080,0.221891,0.013879,1.000000,0.153821,-0.221131,0.059205,0.115679,0.165828,0.034564,-0.011814,-0.088352,-0.125876,0.036205,-0.005801,-0.001964
VRDeck,-0.198857,0.101007,0.125974,-0.019581,0.227995,-0.007322,0.153821,1.000000,-0.207075,0.097108,0.123204,0.208311,-0.003495,-0.018676,-0.095165,-0.128630,0.012239,0.009073,-0.006890
Transported,0.468645,-0.075026,-0.037650,-0.244611,0.046566,0.010141,-0.221131,-0.207075,1.000000,-0.002623,0.144733,0.108193,-0.034046,-0.097965,-0.087753,0.016269,-0.014568,-0.101397,0.101455
A,-0.031378,0.075568,0.134195,-0.023269,0.119257,-0.017418,0.059205,0.097108,-0.002623,1.000000,-0.054651,-0.053409,-0.042018,-0.058312,-0.119881,-0.112510,-0.004179,-0.009344,0.017316


In [234]:
cols_to_fill_nans = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

def use_lr_to_a_columns(df, columns):
    from sklearn.linear_model import LinearRegression
    ys = []
    for column in columns:
        new_df = df[columns]
        test_data = new_df[new_df[column].isnull()]
        new_df.dropna(inplace=True)
        x_train = new_df.drop(column,axis=1)
        y_train = new_df[column]
        lr = LinearRegression()
        lr.fit(x_train, y_train)
        cols_to_test_on = [value for value in columns if value != column]
        x_test = test_data[cols_to_test_on]
        x_test.fillna(0)
        x_test = x_test.replace(np.nan, 0)
        y_pred = lr.predict(x_test)
        ys.append(y_pred)

    return ys

ys = use_lr_to_a_columns(df.copy(), cols_to_fill_nans)

for i, col in enumerate(cols_to_fill_nans): 
        nan_indices = df[df[col].isna()].index
        for index, replacement_value in zip(nan_indices, ys[i]):
            df.at[index, col] = replacement_value

df

/tmp/ipykernel_8785/2533156445.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)
/tmp/ipykernel_8785/2533156445.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)
/tmp/ipykernel_8785/2533156445.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)
/tmp/ipykernel_8785/2533156445.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,A,B,C,D,E,F,G,T,P,S
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,1,0
1,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1,0,0,0,0,0,1,0,0,0,1
2,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0,0,0,0,0,0,0,0,1
3,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0,1,0,0,0,0,0,0,0,0,1
4,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0,1,0,0,0,0,0,0,0,1,0
8689,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,1
8690,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1,0,0,0,0,0,0,1,0,0,1
8691,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0,0,0,0,0,1,0,0,0,0,1


In [235]:
df['Age'] = df['Age'].fillna(df['Age'].mean())

df

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,A,B,C,D,E,F,G,T,P,S
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,1,0
1,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1,0,0,0,0,0,1,0,0,0,1
2,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0,0,0,0,0,0,0,0,1
3,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0,1,0,0,0,0,0,0,0,0,1
4,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0,1,0,0,0,0,0,0,0,1,0
8689,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,1
8690,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1,0,0,0,0,0,0,1,0,0,1
8691,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0,0,0,0,0,1,0,0,0,0,1


In [236]:
def use_lr_to_a_column_2(df, column):
    from sklearn.linear_model import LinearRegression
    test_data = df[df[column].isnull()]
    df.dropna(inplace=True)
    x_train = df.drop(column,axis=1)
    x_train = x_train.drop('Transported',axis=1)
    y_train = df[column]
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    x_test = test_data[x_train.columns]
    y_pred = lr.predict(x_test)
    return y_pred

def fill_nan_with_lr(df):
    df_without_age = df.loc[:, df.columns != 'CryoSleep']
    df_without_cabin = df.loc[:, df.columns != 'VIP']
    a = pd.Series(use_lr_to_a_column_2(df_without_age, 'VIP'))
    b = pd.Series(use_lr_to_a_column_2(df_without_cabin, 'CryoSleep'))

    nan_indices = df[df['VIP'].isna()].index
    for index, replacement_value in zip(nan_indices, a):
        df.at[index, 'VIP'] = replacement_value

    nan_indices = df[df['CryoSleep'].isna()].index
    for index, replacement_value in zip(nan_indices, b):
        df.at[index, 'CryoSleep'] = replacement_value

    return df

df = fill_nan_with_lr(df)
df



/tmp/ipykernel_8785/362490157.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/tmp/ipykernel_8785/362490157.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,A,B,C,D,E,F,G,T,P,S
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,1,0
1,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1,0,0,0,0,0,1,0,0,0,1
2,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0,0,0,0,0,0,0,0,1
3,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0,1,0,0,0,0,0,0,0,0,1
4,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0,1,0,0,0,0,0,0,0,1,0
8689,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,1
8690,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1,0,0,0,0,0,0,1,0,0,1
8691,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0,0,0,0,0,1,0,0,0,0,1


In [237]:
df.isnull().sum()

CryoSleep       0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
A               0
B               0
C               0
D               0
E               0
F               0
G               0
T               0
P               0
S               0
dtype: int64